# **Database Connection Tests**

---

## **1. Imports**

In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import asyncio
import pytest

from db.db_connection import DbConnectionAsync
from db.connection_string import create_connection_string
from utils.config import settings

## **2. Tests**

In [3]:
# Use in-memory SQLite for tests
TEST_CONN_STRING = create_connection_string(
    db_type="mssql",
    database=settings.mssql_db,
    host=settings.host,
    port=int(settings.mssql_port),
    user=settings.user,
    password=settings.mssql_password,
)

In [4]:
@pytest.mark.asyncio
async def test_connect_and_close():
    db = DbConnectionAsync(TEST_CONN_STRING, timeout=1)
    await db.connect()
    assert db.conn is not None
    assert not db.conn.closed

    await db.close()
    assert db.conn.closed

In [5]:
@pytest.mark.asyncio
async def test_execute_creates_connection():
    db = DbConnectionAsync(TEST_CONN_STRING, timeout=1)
    result = await db.execute("SELECT 1 AS x")
    row = result.fetchone()
    assert row[0] == 1
    await db.close()

In [6]:
@pytest.mark.asyncio
async def test_execute_with_params():
    db = DbConnectionAsync(TEST_CONN_STRING)
    await db.connect()

    # Create a table
    await db.execute("CREATE TABLE test (id INTEGER PRIMARY KEY, name TEXT)")
    await db.execute("INSERT INTO test (name) VALUES (:name)", name="Alice")

    # Query back
    result = await db.execute("SELECT name FROM test WHERE name=:name", name="Alice")
    row = result.fetchone()
    assert row[0] == "Alice"

    await db.close()

In [7]:
@pytest.mark.asyncio
async def test_auto_close_after_timeout():
    db = DbConnectionAsync(TEST_CONN_STRING, timeout=1)
    await db.connect()
    assert not db.conn.closed

    # Wait for timeout to trigger
    await asyncio.sleep(1.5)
    assert db.conn is None or db.conn.closed

In [8]:
@pytest.mark.asyncio
async def test_reconnect_after_close():
    db = DbConnectionAsync(TEST_CONN_STRING, timeout=1)
    await db.connect()
    await db.close()

    # Should reconnect automatically on execute
    result = await db.execute("SELECT 42")
    row = result.fetchone()
    assert row[0] == 42
    await db.close()

In [ ]:
@pytest.mark.asyncio
async def test_context_manager():
    async with DbConnectionAsync(TEST_CONN_STRING) as db:
        result = await db.execute("SELECT 99")
        row = result.fetchone()
        assert row[0] == 99

    assert db.conn is None or db.conn.closed

In [10]:
test_connect_and_close()
test_execute_creates_connection()
test_execute_with_params()
test_auto_close_after_timeout()
test_reconnect_after_close()
test_context_manager()

print("✅ All tests ran successfully")

✅ All tests ran successfully


C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:1: RuntimeWarning: coroutine 'test_connect_and_close' was never awaited
  test_connect_and_close()
C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:2: RuntimeWarning: coroutine 'test_execute_creates_connection' was never awaited
  test_execute_creates_connection()
C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:3: RuntimeWarning: coroutine 'test_execute_with_params' was never awaited
  test_execute_with_params()
C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:4: RuntimeWarning: coroutine 'test_auto_close_after_timeout' was never awaited
  test_auto_close_after_timeout()
C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:5: RuntimeWarning: coroutine 'test_reconnect_after_close' was never awaited
  test_reconnect_after_close()
C:\Users\vinh8744\AppData\Local\Temp\ipykernel_12292\994545113.py:6: RuntimeWarning: coroutine 'test_context_manager' was never awa